# Text classification

This week we are moving from  classifiyng characteristics of single words to classifying whole texts. However, instead of trying to classify the sentiment of a text, we will be classifying whether texts are toxic or not. We are using the toxi-text dataset from huggingface. You can find more information about the dataset [here](https://huggingface.co/datasets/FredZhang7/toxi-text-3M). Try to get an overview of:
- what kind of data it contains
- where the data comes from
- what the labels mean

If you prefer not to read toxic text you can use [this](https://huggingface.co/datasets/stanfordnlp/imdb) dataset instead which contains imdb reviews and sentiment classification labels - or any other dataset you prefer :-)

## Install packages

In [ ]:
!pip install nltk
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install gensim
!pip install scikit-learn
!pip install fsspec
!pip install huggingface-hub

## Import packages

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
import gensim.downloader
import numpy as np

## Load data

The dataset is very large and multilingual, so for efficiency's sake we will only use a smaller, English subset of the data. We don't have to split the data into training and test sets because the dataset already has a test set which is saved in a separate file.

In [35]:
df = pd.read_csv("hf://datasets/FredZhang7/toxi-text-3M/train/multilingual-train-deduplicated.csv", nrows=100000)

In [ ]:
df = df[df.lang == 'en']
df

## Preprocessing

The sklearn bag-of-words model expects the data to be a sequence of strings:

In [ ]:
texts = df["text"].tolist()
texts

## Bag-of-words 

One of the simplest way to represent a document is a bag-of-words model. This model represents a document as a set of words, ignoring the order of the words. The model is implemented in the `CountVectorizer` class in sklearn.

In [38]:
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(texts)

In [ ]:
features.shape

The shape of the matrix should correspond to the number of documents and the number of unique words in the dataset. The value of each cell should correspond to the number of times the word appears in the document.

In [ ]:
vectorizer.vocabulary_

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
len(texts)

Lastly, we need to create a list of the labels:

In [45]:
y = df.is_toxic.tolist()

In [ ]:
y

## Training a model

Now we can train a model to classify the toxicity of the texts. I will use a simple logistic regression model, but feel free to swap it out for any other model you prefer.

In [44]:
clf = LogisticRegression(random_state=42)

In [ ]:
clf.fit(features, y)

In [ ]:
clf.score(features, y)

Now try to take a look at the documentation for the [Countvectorizer](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Try to change the parameters of the model and see how it affects the performance of the model:
- try to remove lowercasing and see how it affects performance
- try to add stopwords to the model
- try to see if you can find a parameter that can be used as an alternative to stopword removal
- try to change the ngram_range parameter
- try to change how the model tokenises the text by changing the token_pattern parameter (hint: use a regex generator)

## tf-idf

Another simple, yet slightly more advanced model is the tf-idf model. This model is also implemented in sklearn. The model is implemented in the `TfidfVectorizer` class in sklearn.

- try to create tfidf features from our texts and run the classifier again
- take a look at the [documentation](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and try to change the parameters of the model and see how it affects the performance of the model

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Document embeddings

A much more nuanced way to represent text is through embeddings. However, most machine learning models require a fixed-size input, so we need to find a way to represent the whole document as a fixed-size vector. One way to do this is to use the average of the word embeddings of the words in the document. We will use the pre-trained word embeddings from the GloVe model. However, using word embeddings requires us to split the documents into individual words. We will use the nltk library to do this, but there are both simpler and more advanced ways to do this. The simplest method would be to split the documents by spaces, while a more advanced method would be to use a tokenizer that is aware of the structure of the language, like the one in the [spacy](https://spacy.io/api/tokenizer) library.

In [ ]:
import nltk

nltk.download('punkt')

from nltk.tokenize import word_tokenize

If we try to tokenise the first of the texts, we get:

In [ ]:
word_tokenize(texts[0], language='english', preserve_line=True)

Now we can load the embeddings and match our tokenised words to the embeddings:

In [65]:
embeddings = gensim.downloader.load("glove-wiki-gigaword-300")

In [66]:
def get_embeddings(text):
    return [embeddings[word] for word in word_tokenize(text, language='english', preserve_line=True) if word in embeddings.key_to_index]

In [82]:
text_embeddings = [get_embeddings(text) for text in texts]

In [ ]:
print(len(text_embeddings[0]))
print(len(text_embeddings[0][0]))

In [ ]:
print(len(text_embeddings[1]))
print(len(text_embeddings[1][0]))

We see that though the individual word embeddings have to same number of dimensions, the document embeddings have different sizes. We can fix this by taking the average of the word embeddings:

In [85]:
mean_embeddings = [np.mean(embedding, axis=0) for embedding in text_embeddings]

In [ ]:
mean_embeddings[0].shape

Now you have mean document embeddings that you can use to classify the texts!

- try to classify the texts using the average of the word embeddings of the words in the text
- try lowercasing the words before creating the embeddings
- try removing stopwords or punctuation beore creating the embeddings
- try using another classifier
- try to use all the languages in the dataset and see how it affects the performance of the model